In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC,LinearSVC
from sklearn.metrics import auc, roc_auc_score, mean_squared_error, r2_score, classification_report
from sklearn.ensemble import RandomForestClassifier

import os
import json

In [2]:
pd.set_option('display.max_columns', None)

Loading the data from files:




In [3]:
def construct_data_path(dataset_name):
    """Constructs the path to `dataset_name`.
    
    Parameters
    ----------
    dataset_name: str
        The name of the dataset.
    
    Returns
    -------
    str
        A path to the dataset.
    
    """
    return os.path.join('../output_data', '{}.csv'.format(dataset_name))

In [4]:
train_data_1 = pd.read_csv(construct_data_path('interactions_training'))
validation_data_1 = pd.read_csv(construct_data_path('interactions_validation'))

/home/farid/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
train_data = train_data_1.copy()
validation_data = validation_data_1.copy()

In [6]:
train_data

,user_id,book_id,review_id,is_read,rating,review_text_incomplete,date_added,date_updated,read_at,started_at,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,ratings_count,work_id,title,title_without_series,shelved,read,rated,recommended,year_month_added,year_month_updated,pub_date,is_translated,main_author,is_in_series,series_length,title_description
0,55e33e664f052f3858d8df5d9cd38597,2547,a3a8cf3cead1f647c240ce56d394090d,True,5,NaN,Fri Oct 18 14:39:29 -0700 2013,Fri Oct 18 14:39:29 -0700 2013,NaN,NaN,000100039X,4280,[],US,eng,"[{'count': '1941', 'name': 'poetry'}, {'count'...",NaN,False,4.23,B06XDHQFVY,"['67377', '29946', '700480', '72910', '14572',...","Kahlil Gibran's masterpiece, The Prophet, is o...",Paperback,https://www.goodreads.com/book/show/2547.The_P...,"[{'author_id': '6466154', 'role': ''}]",Rupa & Co,127.0,1.0,9780001000391,1.0,NaN,2010.0,https://www.goodreads.com/book/show/2547.The_P...,https://images.gr-assets.com/books/1355046521m...,145579,2938937,The Prophet,The Prophet,1,1,1,1,2013-10,2013-10,2010-01,0,6466154.0,0,1,"The Prophet Kahlil Gibran's masterpiece, The P..."
1,be813d0bb8f4cd8c4bd77870df3c8138,2696,12774740d9a76a6dc5e2f60289358cf2,True,2,NaN,Tue Feb 07 01:27:13 -0800 2012,Tue Feb 07 01:27:13 -0800 2012,NaN,NaN,0140424385,2071,[],US,eng,"[{'count': '3097', 'name': 'to-read'}, {'count...",NaN,False,3.48,B018HCIHXY,"['3049', '449589', '119079', '51799', '765427'...",The procession that crosses Chaucer's pages is...,Paperback,https://www.goodreads.com/book/show/2696.The_C...,"[{'author_id': '1838', 'role': ''}, {'author_i...",Penguin Classics,504.0,30.0,9780140424386,1.0,NaN,2003.0,https://www.goodreads.com/book/show/2696.The_C...,https://images.gr-assets.com/books/1261208589m...,150099,986234,The Canterbury Tales,The Canterbury Tales,1,1,1,0,2012-02,2012-02,2003-01,0,1838.0,0,1,The Canterbury Tales The procession that cross...
2,695d92b05d837223608e5effcbefa14c,30119,9cc215a3037c26944369c2b839c657fc,True,4,NaN,Fri Apr 23 06:36:42 -0700 2010,Fri Apr 23 06:36:42 -0700 2010,NaN,NaN,0060513039,8950,[],US,eng,"[{'count': '20447', 'name': 'to-read'}, {'coun...",NaN,False,4.29,NaN,"['488240', '420404', '407429', '10065', '83520...",Where the Sidewalk Ends turns forty! Celebrate...,Hardcover,https://www.goodreads.com/book/show/30119.Wher...,"[{'author_id': '435477', 'role': ''}]",Harpercollins Childrens Books,176.0,NaN,9780060513030,11.0,NaN,2002.0,https://www.goodreads.com/book/show/30119.Wher...,https://images.gr-assets.com/books/1168052448m...,1029527,30518,Where the Sidewalk Ends,Where the Sidewalk Ends,1,1,1,1,2010-04,2010-04,2002-11,0,435477.0,0,1,Where the Sidewalk Ends Where the Sidewalk End...
3,841232985badbdc753f6179ab2dcf899,26596,42b2bb4a3396fd2c63ee3fdba160fb03,False,0,NaN,Thu Sep 25 23:17:27 -0700 2014,Thu Sep 25 23:17:28 -0700 2014,NaN,NaN,0871401525,181,[],US,eng,"[{'count': '22872', 'name': 'to-read'}, {'coun...",NaN,False,4.35,NaN,"['142080', '72155', '133380', '367802', '56378...","At the time of his death in 1962, E. E. Cummin...",Hardcover,https://www.goodreads.com/book/show/26596.Comp...,"[{'author_id': '10547', 'role': ''}]",Liveright,1136.0,NaN,9780871401526,NaN,NaN,1994.0,https://www.goodreads.com/book/show/26596.Comp...,https://images.gr-assets.com/books/1355093052m...,18787,175882,"Complete Poems, 1904-1962","Complete Poems, 1904-1962",1,0,0,0,2014-09,2014-09,1994-00,0,10547.0,0,1,"Complete Poems, 1904-1962 At the time of his d..."
4,08ff9ebbf8f94cf1d53b1fd87f077d6b,406373,0ee7eb8e9346ca6805f355fc9d8ad806,True,4,NaN,Tue Sep 18 15:41:21 -0700 2007,Wed Feb 13 07:23:38 -0800 2013,NaN,NaN,0385031149,421,['471367'],US,eng,"[{'count': '425', 'name': 'fiction'}, {'count'...",NaN,False,3.97,B00O4CVNHE,"['505913', '1576915', '849912', '1470', '29376...",Goethe's Faust rew

Step 2: Preprocess the data

    -Remove extra columns 
    -Add Useful columns 
    -Clean the data

In [7]:
def remove_extra_columns(data):
    """remove extra columns.
    
    Parameters
    ----------
    data: DataFrame
        the dataset.
    
    Returns
    -------
    DataFrame
        a dataframe without the extra coloumns.
    
    """
        
    columns_to_keep = ['text_reviews_count', 'popular_shelves', 
                       'average_rating', 'format', 'publisher', 
                       'num_pages', 'publication_month', 'publication_year', 
                       'ratings_count', 'title', 'is_translated', 
                       'is_in_series', 'series_length', 'recommended',
                       ]
#                        'shelved', 'read', 'rated']
    
    return data[columns_to_keep]

In [8]:
def get_popular_values(data, column, count): 
    """finds the top `count` values in `column`.
    
    Parameters
    ----------
    data: DataFrame
        The dataset.
    
    column: str
        Name of the column to search.
        
    count: int
        Number of the popular values to find.
    
    Returns
    -------
    list
        a list of popular values.
    
    """
        
    all_values = data.groupby(data[column])[column].count().sort_values(ascending=False)
    popular_values = all_values[0:count]
    return list(popular_values.index)

In [9]:
def onehot_encode_column(data, column, popular_values):
    """changes the `columns` into a series of one-hot encoded columns. 
    The values that are not in the popular_values will be encoded in the 'other' category. 
    
    Parameters
    ----------
    data: DataFrame
        The dataset.
    
    column: str
        Name of the column to search.
        
    popular_values: list
        A list of the popular_values in the `column` that will be kept. 
    
    Returns
    -------
    DataFrame
        a dataframe with the one-hot encoded columns. 
    
    """
        
    data = data.copy()
    data.loc[~(data[column].isin(popular_values)), column] = 'other'
    data = data.join(pd.get_dummies(data[column], prefix=column))
    data = data.drop([column],axis=1, errors='ignore')    
    return data

In [10]:
def get_total_shelved_count(shelves_str):
    """Find the total number of shelves that a book is in.
    
    Parameters
    ----------
    str: shelves_str
        a json encoded dictionary contrining the shelving information.
    
    Returns
    -------
    int
        total number of shelves a book is in.
    
    """
    
    total = 0
    shelves_str = shelves_str.replace("\'", "\"")
    shelves_list = json.loads(shelves_str)
    for shelve in shelves_list:
        total += int(shelve['count'])
    return total

In [11]:
def get_title_len(row):
    """Get the length of the title of the book. 
    
    Parameters
    ----------
    DataFrame Row: row
        A row of DataFrame corresponding the the book 
    
    Returns
    -------
    int
        The length of the title. Returns 0 on failure 
    
    """
        
    try:
        return len(row['title'])
    except:
        return 0

In [12]:
def engineer_features(data, popular_formats, popular_publishers):
    """Removes the extra features and adds useful ones.  
    
    Parameters
    ----------
    DataFrame: data
        A DataFrame
        
    popular_formats: list
        A list of the popular 'formats' that will be kept. 
    
    popular_publishers: list
        A list of the popular 'publishers' that will be kept. 
        
    
    
    Returns
    -------
    DataFrame:
        A modified DataFrame with one-hot encoded format and publishers, 
        total number of shelved instances, length of the title of the book, 
        and 0-filled NaN values in numeric fields. 
        Also without extra columns and without non-numeric fields. 
    
    """
        
        
    data = data.copy()
    
    data = remove_extra_columns(data)

    data = onehot_encode_column(data, 'format', popular_formats)
    data = onehot_encode_column(data, 'publisher', popular_publishers)
    
    data['shelved_count'] = data.apply(lambda row: get_total_shelved_count(row['popular_shelves']), axis=1)
    data['title_len'] = data.apply(lambda row: get_title_len(row), axis=1)

    fillna_columns = ['publication_month','publication_year',
                      'text_reviews_count','average_rating',
                      'num_pages','ratings_count']
    data[fillna_columns] = data[fillna_columns].fillna(0)

        
    data = data.drop(['popular_shelves','title'],axis=1)
    
    return data

In [13]:
popular_formats = get_popular_values(train_data,'format', 5)
popular_publishers = get_popular_values(train_data, 'publisher', 5)

In [14]:
train_data = engineer_features(train_data, popular_formats, popular_publishers)
validation_data = engineer_features(validation_data, popular_formats, popular_publishers)

In [15]:
train_columns = train_data.columns.tolist()
validation_data = validation_data.reindex(columns=train_columns, fill_value=0)

X_train = train_data.drop('recommended', axis = 1)
y_train = train_data['recommended']

X_validation = validation_data.drop('recommended', axis = 1)
y_validation = validation_data['recommended']

In [16]:
X_train.sample(10)

,text_reviews_count,average_rating,num_pages,publication_month,publication_year,ratings_count,is_translated,is_in_series,series_length,format_Hardcover,format_Kindle Edition,format_Mass Market Paperback,format_Paperback,format_ebook,format_other,publisher_Cambridge University Press,publisher_Createspace,publisher_Harpercollins Childrens Books,publisher_Penguin Classics,publisher_Vintage,publisher_other,shelved_count,title_len
168406,14,3.82,88.0,3.0,2002.0,313,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,490,3
14318,709,4.50,544.0,12.0,2002.0,9396,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,4278,20
82262,1104,4.14,56.0,1.0,2001.0,73977,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,23913,20
27661,5737,4.01,289.0,8.0,2005.0,526122,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,12786,6
60904,55,4.45,288.0,2.0,1998.0,932,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1208,51
119269,27,4.06,0.0,2.0,1999.0,7145,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,3970,6
176530,26,2.63,12.0,0.0,0.0,38,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,86,27
48408,351,4.19,476.0,8.0,2003.0,5671,0,1,2,0,0,0,1,0,0,0,0,0,0,0,1,5878,10
26771,5737,4.01,289.0,8.0,2005.0,526122,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,12786,6
22016,40,4.05,110.0,12.0,1993.0,4889,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,2257,31


Step 3: Prediction

In [17]:
def predict_logistic_regression(X_test, X_train, y_train):
    """classify X_test entries with a logistic regression classifier 
    
    Parameters
    ----------
    DataFrame: X_test
        A DataFrame corresponding to the test data
        
    DataFrame: X_train
        A DataFrame corresponding to the train data
        
    list: y_train
        A list of categories corresponding to the Xs in the X_train
        
    
    
    Returns
    -------
    list: 
        A list of predicted categories corresponding the X_test
    
    """
        
    regr = LogisticRegression(max_iter = 10000)
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    
    return y_pred

In [20]:
def predict_svm(X_test, X_train, y_train):
    """classify X_test entries with a logistic a linear support vector classifier 
    
    Parameters
    ----------
    DataFrame: X_test
        A DataFrame corresponding to the test data
        
    DataFrame: X_train
        A DataFrame corresponding to the train data
        
    list: y_train
        A list of categories corresponding to the Xs in the X_train
        
    
    
    Returns
    -------
    list: 
        A list of predicted categories corresponding the X_test
    
    """
        
    
    clf = make_pipeline(StandardScaler(), 
                        LinearSVC(random_state=0, max_iter=10000))
    
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    return y_pred

In [35]:
def predict_random_forest(X_test, X_train, y_train):
    """classify X_test entries with a Random Forest classifier 
    
    Parameters
    ----------
    DataFrame: X_test
        A DataFrame corresponding to the test data
        
    DataFrame: X_train
        A DataFrame corresponding to the train data
        
    list: y_train
        A list of categories corresponding to the Xs in the X_train
        
    
    
    Returns
    -------
    list: 
        A list of predicted categories corresponding the X_test
    
    """
        
    clf = RandomForestClassifier(n_jobs=4, n_estimators=1000, max_depth=30, random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred

In [33]:
def test_model(model, X_test, y_test, X_train, y_train):
    y_pred_test = model(X_test, X_train, y_train)
    y_pred_train = model(X_train, X_train, y_train)
    
    print("validation:")
    print(classification_report(y_test, y_pred_test))
    print(roc_auc_score(y_test, y_pred_test))
    
    print("train:")
    print(classification_report(y_train, y_pred_train))
    print(roc_auc_score(y_train, y_pred_train))
    

In [26]:
test_model(predict_logistic_regression, X_validation, y_validation, X_train, y_train)

validation:
              precision    recall  f1-score   support

           0       0.57      0.60      0.59     17586
           1       0.57      0.54      0.55     17044

    accuracy                           0.57     34630
   macro avg       0.57      0.57      0.57     34630
weighted avg       0.57      0.57      0.57     34630

0.5692610095563364
train:
              precision    recall  f1-score   support

           0       0.55      0.67      0.61    111170
           1       0.70      0.59      0.64    146475

    accuracy                           0.62    257645
   macro avg       0.63      0.63      0.62    257645
weighted avg       0.64      0.62      0.63    257645

0.6299177778327409


In [36]:
test_model(predict_random_forest, X_validation, y_validation, X_train, y_train)

validation:
              precision    recall  f1-score   support

           0       0.66      0.49      0.56     17586
           1       0.58      0.74      0.65     17044

    accuracy                           0.61     34630
   macro avg       0.62      0.62      0.61     34630
weighted avg       0.62      0.61      0.61     34630

0.6155739449514643
train:
              precision    recall  f1-score   support

           0       0.67      0.58      0.62    111170
           1       0.71      0.78      0.74    146475

    accuracy                           0.69    257645
   macro avg       0.69      0.68      0.68    257645
weighted avg       0.69      0.69      0.69    257645

0.6803388611102169


In [37]:
test_model(predict_svm, X_validation[:10000], y_validation[:10000], X_train[:50000], y_train[:50000])

validation:
              precision    recall  f1-score   support

           0       0.57      0.57      0.57      5095
           1       0.56      0.56      0.56      4905

    accuracy                           0.57     10000
   macro avg       0.57      0.57      0.57     10000
weighted avg       0.57      0.57      0.57     10000

0.5658660776540331
train:
              precision    recall  f1-score   support

           0       0.56      0.64      0.59     21495
           1       0.69      0.62      0.65     28505

    accuracy                           0.63     50000
   macro avg       0.63      0.63      0.62     50000
weighted avg       0.64      0.63      0.63     50000

0.6281824228304523
